In [1]:
from morty.extras.foldgenerator import FoldGenerator
from morty.classifiers.knnclassifier import KNNClassifier
from morty.evaluator import Evaluator
import numpy as np
import json
import os

In [2]:
# data
data_folder = 'data'
annotation_file = os.path.join(data_folder, 'annotations.json')
annotations = json.load(open(annotation_file, 'r'))

In [3]:
# divide the data into stratified 5-fold
random_state = 1  # for reproducability
folds = FoldGenerator.stratified_k_fold(data_folder, annotations, n_folds=5, 
                                        random_state=random_state)

In [4]:
# instantiate the classifier object
step_size = 7.5
kernel_width = 15
model_type = 'multi'
distribution_type = "pcd"
distance_method = "bhat"
rank = 2
k_neighbor = 15
min_peak_ratio = 0.15
classifier = KNNClassifier(step_size=step_size, kernel_width=kernel_width, 
                           feature_type=distribution_type)
ev = Evaluator()

In [5]:
# train, test and evaluate the method for each fold
tonic_res = []
mode_res = []
joint_res = []
for fold in folds:
    classifier.train(model_type=model_type, **fold['training'])
    
    # testing
    testing_data = fold['testing']
    for t in testing_data:          
        # tonic identification
        tonic = classifier.estimate_tonic(
            t['pitch'], t['mode'], min_peak_ratio=min_peak_ratio, 
            distance_method=distance_method, k_neighbor=k_neighbor, rank=rank)
        
        # mode recognition
        mode = classifier.estimate_mode(
            t['pitch'], t['tonic'], distance_method=distance_method, 
            k_neighbor=k_neighbor, rank=rank)

        # joint estimation
        joint = classifier.estimate_joint(
            t['pitch'], min_peak_ratio=min_peak_ratio, distance_method=distance_method, 
            k_neighbor=k_neighbor, rank=rank)
        
        # evaluate
        tonic_res.append(ev.evaluate_tonic(tonic[0][0], t['tonic'], t['source']))
        mode_res.append(ev.evaluate_mode(mode[0][0], t['mode'], t['source']))
        joint_res.append(ev.evaluate_joint([joint[0][0][0], t["tonic"]], 
                                           [joint[0][0][1], t["mode"]], t['source']))
        
        # display results
        print(u"mbid: {0:s}".format(t["source"]))
        print(u"Tonic Iden: Est: {0:.1f}Hz\t Anno: {1:.1f}Hz\tEval: {2:s}"
              .format(tonic[0][0], t["tonic"], str(tonic_res[-1]['tonic_eval'])))
        print(u"Mode Recog: Est: {0:8.8s}\t Anno: {1:8.8s}\tEval: {2:s}"
              .format(mode[0][0], t["mode"], str(mode_res[-1]['mode_eval'])))
        print(u"Joint Estim:\n"
              u"\tTonic Iden: Est: {0:.1f}Hz\t Anno: {1:.1f}Hz\tEval: {2!r}\n"
              u"\tMode Recog: Est: {3:8.8s}\t Anno: {4:8.8s}\tEval: {5!r}\n"
              .
              format(joint[0][0][0], t['tonic'], joint_res[-1]['tonic_eval'],
                     joint[0][0][1], t['mode'], joint_res[-1]['mode_eval']))
        print("----------------------------------------------------------------------")

mbid: 0f0e4bc3-67f9-4727-818b-983320e897cb
Tonic Iden: Est: 451.6Hz	 Anno: 225.6Hz	Eval: True
Mode Recog: Est: Saba    	 Anno: Saba    	Eval: True
Joint Estim:
	Tonic Iden: Est: 451.6Hz	 Anno: 225.6Hz	Eval: True
	Mode Recog: Est: Saba    	 Anno: Saba    	Eval: True

----------------------------------------------------------------------
mbid: 0db48ce4-f018-4d7d-b75e-66a64db72067
Tonic Iden: Est: 303.1Hz	 Anno: 151.1Hz	Eval: True
Mode Recog: Est: Hicaz   	 Anno: Hicaz   	Eval: True
Joint Estim:
	Tonic Iden: Est: 303.1Hz	 Anno: 151.1Hz	Eval: True
	Mode Recog: Est: Hicaz   	 Anno: Hicaz   	Eval: True

----------------------------------------------------------------------
mbid: 0eac190d-13c4-442f-bb13-cf734d3cbe88
Tonic Iden: Est: 441.9Hz	 Anno: 221.0Hz	Eval: True
Mode Recog: Est: Hicaz   	 Anno: Huseyni 	Eval: False
Joint Estim:
	Tonic Iden: Est: 664.0Hz	 Anno: 221.0Hz	Eval: False
	Mode Recog: Est: Hicaz   	 Anno: Huseyni 	Eval: False

------------------------------------------------------

In [6]:
# get overall results 
tonic_accuracy = np.mean([b['tonic_eval'] for b in tonic_res])
mode_accuracy = np.mean([b['mode_eval'] for b in mode_res])
joint_accuracy = np.mean([b['joint_eval'] for b in joint_res])

# display
print("Tonic Accuracy: %.2f" %tonic_accuracy)
print("Mode Accuracy : %.2f" %mode_accuracy)
print("Joint Accuracy: %.2f" %joint_accuracy)

Tonic Accuracy: 0.93
Mode Accuracy : 0.93
Joint Accuracy: 0.93
